In [6]:
import xarray as xr
import matplotlib.pyplot as plt
import cmocean.cm as cm
import copy
from salishsea_tools import viz_tools
import pandas as pd
import netCDF4 as nc

In [3]:
data = pd.read_csv('/home/sallen/MEOPAR/Metro_Data/UBC Output.xlsx-2023-7-31 15.19.6.csv',low_memory=False, parse_dates=[11])
data.keys()
data_time = data.set_index('Sampled Date')
data_time['LIMS Analysis Code'].unique()

array(['PH', 'TEMP', 'BOD_CALC', 'COD', 'SS', 'COND', 'VSS', 'NH3', 'CL',
       'HARDNESS', 'SO4', 'TKN', 'AL_DISSOLVED', 'BA_DISSOLVED',
       'B_DISSOLVED', 'CD_DISSOLVED', 'CR_DISSOLVED', 'CO_DISSOLVED',
       'CU_DISSOLVED', 'FE_DISSOLVED', 'PB_DISSOLVED', 'MN_DISSOLVED',
       'MO_DISSOLVED', 'NI_DISSOLVED', 'P_DISSOLVED', 'AG_DISSOLVED',
       'SN_DISSOLVED', 'ZN_DISSOLVED', 'AL_TOTAL', 'AS_TOTAL', 'BA_TOTAL',
       'B_TOTAL', 'CD_TOTAL', 'CA_TOTAL', 'CR_TOTAL', 'CO_TOTAL',
       'CU_TOTAL', 'FE_TOTAL', 'PB_TOTAL', 'MG_TOTAL', 'MN_TOTAL',
       'HG_TOTAL', 'MO_TOTAL', 'NI_TOTAL', 'P_TOTAL', 'SE_TOTAL',
       'AG_TOTAL', 'SN_TOTAL', 'ZN_TOTAL', 'DO', 'NO3', 'NO2', 'CN_TOTAL',
       'ALKALINITY', 'CA_DISSOLVED', 'MG_DISSOLVED', 'K_DISSOLVED',
       'NA_DISSOLVED', 'K_TOTAL', 'NA_TOTAL', 'DOC', 'TOC', 'TEMP_INSITU',
       'PH_INSITU', 'DO_INSITU', 'AS_DISSOLVED', 'SE_DISSOLVED', 'F',
       'PH_15_DEGREES', 'UN-IONIZED_NH3', 'SB_TOTAL', 'BE_TOTAL',
       'BI_TOTAL', 'LI

In [19]:
def filterData(field):
    data_F = data_time.loc[data_time['LIMS Analysis Code']== field]
    data_F = data_F[~data_F.index.isna()]
    data_F['Result'] = pd.to_numeric(data_F.Result, errors = 'coerce')
    data_F_Result = data_F.Result
    result = data_F_Result.groupby(data_F_Result.index.month).mean()
    return result

In [32]:
output_nc_file = "output.nc"
nc_file = nc.Dataset(output_nc_file, "r+", format="NETCDF4")
nc_file.createDimension("time", 12)

<class 'netCDF4._netCDF4.Dimension'>: name = 'time', size = 12

In [33]:
temp = filterData('TEMP')
value = temp.values
index = temp.index

In [34]:
temp_var = nc_file.createVariable('temp', 'f4', ('time',))
temp_var.units = 'degree_C'
temp_var[:] = value
temp_var.long_name = 'Temperature'
nc_file.close()

In [35]:
data = xr.open_dataset('/ocean/jtao/MOAD/analysis-jake/notebooks/output.nc')
data.temp


<xarray.DataArray 'temp' (time: 12)>
[12 values with dtype=float32]
Dimensions without coordinates: time
Attributes:
    units:      degree_C
    long_name:  Temperature